In [ ]:
### VectorStoreInex를 어떻게 선언하는지 숙지하기E ###

city_names = ['Berlin', 'Shanghai', 'Los Angeles', 'Seoul', 'Saint Petersburg', 
              'Beijing', 'Rio de Janeiro', 'Mexico City', 'Budapest', 'Barcelona', 
              'Antananarivo', 'Suwon', 'Belize City']

reader = WikipediaReader()
documents = reader.load_data(city_names, auto_suggest=False)
index = VectorStoreIndex.from_documents(documents) # 외워야 할 듯
query_engine = index.as_query_engine()

In [ ]:
### LLM에 보내는 프롬프트는 기본적으로 요런 구조 
    여기에 f" ... " 이런걸 추가해서 보낸다.
###
messages = [
        { "role": "system", "content": "You are a helpful assistant."},
        { "role": "user",   "content": question, },
    ]



In [ ]:
### YOUR CODE HERE ###
### 복사하긴 했지만 형태만 약간 다를뿐 위에 있는 generate_response와 거의 유사한 내용 ###

def generate_answer(question):
    messages = [
        {
            "role": "user",
            "content": question,
        },
    ]

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )

    return response.choices[0].message.content

In [ ]:
unique_domains = {}
for item in dataset: # dataset을 순회하면서,
    if 'domain' in item: # 각 item에 'domain' 키가 있으면 그 값을 기준으로
        domain_value = item['domain']
        if domain_value not in unique_domains: # 고유한 도메인별 첫 번째 item만 저장하는 로직
            unique_domains[domain_value] = item

# 이런식으로 사용할수도 있음. 결국은 키값을 기준으로 샘플 만들어 내는거
unique_dynamism = {}
for item in dataset:
    if 'static_or_dynamic' in item:
        question_type = item['static_or_dynamic']
        if question_type not in unique_dynamism:
            unique_dynamism[question_type] = item


In [ ]:
all_chunks = []

for html_text in example_data['search_results']:
    soup = BeautifulSoup(html_text["page_result"], features="lxml")
    text = soup.get_text(" ", strip=True)
    # HTML 태그를 제거하고 순수 텍스트만 추출
    if not text:
        all_chunks.append("")
    else:
        _, offsets = text_to_sentences_and_offsets(text)
        # 텍스트를 문장 단위로 나누고, 각 문장의 시작과 끝 위치(offset)를 계산
        # offsets는 문장별로 (start, end) 인덱스를 담고 있는 리스트

        chunks = []

        for start, end in offsets:
            chunk = text[start:end][:4000] # 문자열이 너무 길 경우 최대 4000자까지만 유지
            all_chunks.append(chunk)
